In [1]:
import requests
import time
import random
import pandas as pd
import json
import math

**Crawl data product**

In [2]:
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referrer': 'https://tiki.vn/nha-sach-tiki/c8322',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'X-Guest-Token': 'EtZd0Tml8y4I3kxv5uonOhPsSXGR9bz6',
    'Content-Type': 'application/json',
    'Via': '1.1 google',
    'X-Content-Type-Options': 'nosniff'
}
params = {
    'limit': 40,
    'include': 'advertisement',
    'page': 1,
    'aggregations': 2,
    'trackity_id': 'd3a5d230-b5b5-b643-f1ab-eed0b6703194',
    'category': 8322,
    'page': 1,
    'urlKey': 'nha-sach-tiki'
}

def parse_res(record): 
    d = dict()
    d['id'] = record.get('id')
    d['name'] = record.get('name')
    d['price'] = record.get('price')
    d['original_price'] = record.get('original_price')
    d['discount_rate'] = record.get('discount_rate')
    try: d['quantity_sold'] = record.get('quantity_sold').get('value')
    except: d['quantity_sold'] = 0
    d['rating_average'] = record.get('rating_average')
    d['review_count'] = record.get('review_count')
    d['seller_id'] = record.get('seller_id')
    d['category'] = record.get('visible_impression_info').get('amplitude').get('primary_category_name')
    d['spid'] = record.get('seller_product_id')
    return d

def fetch_products(headers, params):
    products = list()
    for i in range(1,51):
        params['page'] = i
        response = requests.get('https://tiki.vn/api/personalish/v1/blocks/listings?', headers=headers, params=params)
        if response.status_code == 200:
            print('Request Success!: Page {}'.format(i))
            for record in response.json().get('data'):
                obj = parse_res(record)
                products.append(obj)
    
    return products

def main():    
    products = fetch_products(headers, params)
    df = pd.DataFrame(products)
    df.to_csv('products.csv', index=False)
    
if __name__ == '__main__':
    main()

Request Success!: Page 1
Request Success!: Page 2
Request Success!: Page 3
Request Success!: Page 4
Request Success!: Page 5
Request Success!: Page 6
Request Success!: Page 7
Request Success!: Page 8
Request Success!: Page 9
Request Success!: Page 10
Request Success!: Page 11
Request Success!: Page 12
Request Success!: Page 13
Request Success!: Page 14
Request Success!: Page 15
Request Success!: Page 16
Request Success!: Page 17
Request Success!: Page 18
Request Success!: Page 19
Request Success!: Page 20
Request Success!: Page 21
Request Success!: Page 22
Request Success!: Page 23
Request Success!: Page 24
Request Success!: Page 25
Request Success!: Page 26
Request Success!: Page 27
Request Success!: Page 28
Request Success!: Page 29
Request Success!: Page 30
Request Success!: Page 31
Request Success!: Page 32
Request Success!: Page 33
Request Success!: Page 34
Request Success!: Page 35
Request Success!: Page 36
Request Success!: Page 37
Request Success!: Page 38
Request Success!: Pag

**Crawl thêm data về seller và author của product**

In [3]:
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referrer': 'https://tiki.vn/nha-sach-tiki/c8322',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'X-Guest-Token': 'dC1Xx9k2aj3EeWwPoIuySHRUshJDVTl7',
    'Content-Type': 'application/json; charset=utf-8',
    'Via': '1.1 google',
    'X-Content-Type-Options': 'nosniff'
}

params = {
    'platform': 'web',
    'spid': 35191894,
    'version': 3
}


def fetch_sellers_authors(headers, params):
    products = pd.read_csv('products.csv')
    pid_list = products[['id', 'spid']]

    seller_list = list()
    author_list = list()
    i = 1
    count = 1
    
    for index, item in pid_list.iterrows():
        params['spid'] = item[1]
        response = requests.get('https://tiki.vn/api/v2/products/{}?'.format(item[0]), headers=headers, params=params)
        time.sleep(0.4)
        if response.status_code == 200 and 'application/json' in response.headers['Content-Type']:
            print('Request: {}, PID: {} - SPID {}'.format(i,item[0],item[1]), count)
            current_seller = response.json().get('current_seller')
            if current_seller != None:
                seller_list.append(current_seller.get('name'))
            else: seller_list.append('')

            try: author = response.json().get('authors')[0].get('name')
            except: author = ''
            author_list.append(author)
        else: 
            seller_list.append('')
            author_list.append(author)
            count += 1
        i = i + 1    
    return seller_list, author_list

def main():
    sellers, authors = fetch_sellers_authors(headers, params)
    products = pd.read_csv('products.csv')
    products['seller_name'] = sellers
    products['author_name'] = authors
    products.to_csv('products.csv', index=False)

if __name__ == '__main__':
    main()
    # print(sellers)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_16832\713313750.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params['spid'] = item[1]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16832\713313750.py:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  response = requests.get('https://tiki.vn/api/v2/products/{}?'.format(item[0]), headers=headers, params=params)
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16832\713313750.py:34: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value

Request: 1, PID: 74021317 - SPID 74021318 1
Request: 2, PID: 249009518 - SPID 249009520 1
Request: 3, PID: 273612826 - SPID 273612827 1
Request: 4, PID: 273536931 - SPID 273536932 1
Request: 5, PID: 182784585 - SPID 182784586 1
Request: 6, PID: 271379451 - SPID 271379452 1
Request: 7, PID: 262977989 - SPID 262977990 1
Request: 8, PID: 187827003 - SPID 187827005 1
Request: 9, PID: 273401813 - SPID 162140223 1
Request: 10, PID: 74021317 - SPID 74021318 1
Request: 11, PID: 180584741 - SPID 180584742 1
Request: 12, PID: 271442564 - SPID 271442572 1
Request: 13, PID: 272412121 - SPID 271362923 1
Request: 14, PID: 270704495 - SPID 270704496 1
Request: 15, PID: 271405604 - SPID 271405605 1
Request: 16, PID: 195291930 - SPID 195291931 1
Request: 17, PID: 271916321 - SPID 271916323 1
Request: 18, PID: 42230121 - SPID 42230122 1
Request: 19, PID: 212356652 - SPID 212356653 1
Request: 20, PID: 273321955 - SPID 273321956 1
Request: 21, PID: 145301857 - SPID 145301858 1
Request: 22, PID: 262904946 

**Crawl các review về sản phẩm**

In [4]:
headers = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Referrer': 'https://tiki.vn/nha-sach-tiki/c8322',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
    'X-Guest-Token': '1HdCx3rAeKhulntzMOLSjD9sIPXfYy8m',
    'Content-Type': 'application/json; charset=utf-8',
    'Via': 'kong/2.1.4, 1.1 google',
    'X-Content-Type-Options': 'nosniff'
}

params = {
    'limit': 20,
    'include': 'comments,contribute_info,attribute_vote_summary',
    'sort': 'score|desc,id|desc,stars|all',
    'page': 1,
    'spid': 187827005,
    'product_id': 187827003,
    'seller_id': 1
}

def parse_review(record): 
    d = dict()
    d['id'] = record.get('id')
    d['created_at'] = record.get('created_at')
    d['rating'] = record.get('rating')
    d['title'] = record.get('title')
    d['content'] = record.get('content')
    d['thank_count'] = record.get('thank_count')
    try: d['customer_name'] = record.get('created_by').get('full_name')
    except:d['customer_name'] = ''
    d['customer_id'] = record.get('customer_id')
    d['product_id'] = record.get('product_id')
    d['seller_product_id'] = record.get('spid')
    return d

def fetch_reviews(headers, params):
    products = pd.read_csv('products.csv')
    pid_list = products
    ii = 1
    tt = 0
    review_list = list()
    # try:
    for index, item in pid_list[['id', 'spid', 'seller_id', 'review_count']].iterrows():
        params['product_id'] = item[0]
        params['spid'] = item[1]
        params['seller_id'] = item[2]
        max_page = math.ceil(item[3]/20) # 5 is default limit in params, 20 is max.
        count = 0
        for i in range(1,max_page + 1):
            params['page'] = i
            response = requests.get('https://tiki.vn/api/v2/reviews?', headers=headers, params=params)
            time.sleep(0.2)
            if response.status_code == 200 and 'application/json' in response.headers['Content-Type']:
                # print('Request Success: PID - {}, SPID - {}, SELLER_ID - {}, PAGE - {}'.format(params['product_id'],params['spid'],params['seller_id'],params['page']))
                count+=1
                reviews = response.json().get('data')
                for record in reviews:
                    obj = parse_review(record)
                    review_list.append(obj)
        if max_page!=count: tt+=1
        print('products', ii, tt)
        ii += 1
        
    
    return review_list
def main():
    reviews = fetch_reviews(headers, params)
    df = pd.DataFrame(reviews)
    df.to_csv('reviews.csv', index=False)
    
if __name__ == '__main__':
    main()

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16832\3127838178.py:46: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params['product_id'] = item[0]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16832\3127838178.py:47: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params['spid'] = item[1]
C:\Users\ASUS\AppData\Local\Temp\ipykernel_16832\3127838178.py:48: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  params['seller_id'] = item[2]
C:\Users\AS

products 1 0
products 2 0
products 3 0
products 4 0
products 5 0
products 6 0
products 7 0
products 8 0
products 9 0
products 10 0
products 11 0
products 12 0
products 13 0
products 14 0
products 15 0
products 16 0
products 17 0
products 18 0
products 19 0
products 20 0
products 21 0
products 22 0
products 23 0
products 24 0
products 25 0
products 26 0
products 27 0
products 28 0
products 29 0
products 30 0
products 31 0
products 32 0
products 33 0
products 34 0
products 35 0
products 36 0
products 37 0
products 38 0
products 39 0
products 40 0
products 41 0
products 42 0
products 43 0
products 44 0
products 45 0
products 46 0
products 47 0
products 48 0
products 49 0
products 50 0
products 51 0
products 52 0
products 53 0
products 54 0
products 55 0
products 56 0
products 57 0
products 58 0
products 59 0
products 60 0
products 61 0
products 62 0
products 63 0
products 64 0
products 65 0
products 66 0
products 67 0
products 68 0
products 69 0
products 70 0
products 71 0
products 72 0
p